Item 15 Be Cautious When Relying on dict Insertion Ordering      

Things to Remember
- Since Python 3.7, you can rely on the fact that iterating a dict instance's contents will occur in the same ordr in which the keys were initially added
- Python makes it easy to define objects that act like dictionaries but that aren't dict instances. For these types, you can't assume that insertion ordering will be preserved
- There are three ways to be careful about dictionary-like classes: Write code that doesn't rely on insertion ordering, explicitly check for the dict type at runtime, or require dict values using type annotations and static analysis 

In [ ]:
# dictionaries will preserve insertion order in Python 3.7 and above but not so in Python 3.5
baby_names = {
    'cat': 'kitten',
    'dog': 'puppy'
}
print(baby_names)
print(list(baby_names.keys()))
print(list(baby_names.values()))
print(list(baby_names.items()))
print(list(baby_names.popitem())) # last item inserted

In [ ]:
# **kwargs catch-all parameter 
# the order of keyword arguments is always preserved to
# match how the programmer originally called the function (Python 3.7)
def my_func(**kwargs):
    for key, value in kwargs.items():
        print('%s = %s' % (key, value))
my_func(goose='gosling', kangaroo='joey') # order is preserved (3.7) 

In [ ]:
# classes use the dict type for their instance dictionaries - order is preserved (3.7)
class MyClass:
    def __init__(self):
        self.alligator = 'hatchling'
        self.elephant = 'calf'
a = MyClass()
for key, value in a.__dict__.items():
    print(f'{key} = {value}') # alligator then elephant


- Python is not statically typed, so most code relies on duck typing
- Duck typing is similar to, but distinct from, structural typing. Structural typing is a static typing system that determines type compatibility and equivalence by a type's structure, whereas duck typing is dynamic and determines type compatibility by only that part of a type's structure that is accessed during run time.
- So programmers can easily define their own custom container types that emulate the standard protocols matching list, dict, and other types.
- This can result in surprising gotchas as the order might not be preserved when you iterate through the custom container types. 

In [ ]:
votes = {
    'otter': 1281,
    'polar bear': 587,
    'fox': 863
}

# process voting data and save the rank of each animal name into
# a provided empty dictionary 
def populate_ranks(votes, ranks):
    names = list(votes.keys())
    names.sort(key=votes.get, reverse=True) # order by values descending
    for i, name in enumerate(names, 1):
        ranks[name] = i

# this function assumes that the dictionary's iteration is in 
# insertion order to match populate_ranks - highest rank first
def get_winner(ranks):
    return next(iter(ranks))


In [ ]:
ranks = {}
populate_ranks(votes, ranks)
print(ranks)
winner = get_winner(ranks)
print(winner) # all good so far as it's dealing with a true dict

In [ ]:
# now the custom container type
from collections.abc import MutableMapping
class SortedDict(MutableMapping):
    def __init__(self):
        self.data = {}
    def __getitem__(self, key):
        return self.data[key]
    def __setitem__(self, key, value):
        self.data[key] = value
    def __delitem__(self, key):
        del self.data[key]
    def __iter__(self):
        keys = list(self.data.keys())
        keys.sort() # order by key ascending; deviate from insertion order
        for key in keys:
            yield key
    def __len__(self):
        return len(self.data)


In [ ]:
sorted_ranks = SortedDict()
populate_ranks(votes, sorted_ranks)
print(sorted_ranks.data) # the insertion order is preserved here as data is a dict
# not so when dealing with iterator; 'fox' is returned insted  
winner = get_winner(sorted_ranks) 
print(winner)

In [ ]:
# solution 1 - not assuming that the ranks dictionary has a specific iteration order
def get_winner(ranks):
    for name, rank in ranks.items():
        if rank == 1:
            return name
winner = get_winner(sorted_ranks)
print(winner) # otter

In [ ]:
# solution 2 - make sure the ranks parameter is a dict
def get_winner(ranks):
    if not isinstance(ranks, dict):
        raise TypeError('must provide a dict instance')
    return next(iter(ranks))

get_winner(sorted_ranks) # exception